In [ ]:
import urllib
import urllib.request
import os

if not os.path.exists('KRProject_1.zip'):
    !wget https://raw.githubusercontent.com/UvA-KR16/KilerSudoku/master/data/KRProject_1.zip

import zipfile
if not os.path.exists('KRProject_1'):
    dir_name = 'KRProject_1'
    os.mkdir(dir_name)
    zip_handler = zipfile.ZipFile('KRProject_1.zip', "r")
    zip_handler.extractall(dir_name)

In [ ]:
for dir, dirs, files in os.walk('KRProject_1/'):
    for file in files:
        if not file.endswith('.killer'): continue
            
        cells = {}
        for line in open(dir+'/'+file).read().split('\n'):
            if line.strip() == "": continue
            sm = int(line.split('=')[0].strip())
            vals = [(int(j.split(',')[0].strip()), int(j.split(',')[1].strip())) for j in [i.split(')')[0] for i in line.split('(')[1:]]]
            
            cells[sm, vals[0]] = vals

        weburl = 'https://github.com/UvA-KR16/KilerSudoku/tree/master/data/'+os.path.split(dir)[-1]+'/'+file
        txt = '#source: '+weburl+'\n'
        txt += '+'+('---+'*9)+'\n'
        for i in range(0,9):
            txt += '|'
            markers = []
            for j in range(0,9):
                isin = False
                for (sm, fval), vals in cells.items():
                    if (i,j) in vals:
                        break
                    
                txt += (str(sm)+'   ')[:3] if fval == (i,j) else '   '
                isthick = (j+1)%3==0 and j<8 
                txt += ('|║')[isthick] if (i,j+1) not in vals else '  '[isthick]
                markers.append(not (i+1,j) in vals)
            txt += '\n'
            isthick = (i+1)%3==0 and i<8
            txt += '+' + '+'.join(('---','===')[isthick] if i else ('   ','   ')[isthick] for i in markers) + '+\n'

        filename = '../killersudoku-collection/'+os.path.split(dir)[-1]+'/'+file[:-7]+'.txt'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        open(filename, 'w').write(txt)
        

In [ ]:
#--------------------------------------------------
# convert to emdw readables for emdw side of things
#--------------------------------------------------

import sys
import os
if not '..' in sys.path: sys.path.append("..")
import killersudoku

dirs = set()
for dir, _, files in os.walk('../killersudoku-collection/'):
    for file in files:
        file = dir+'/'+file
        cages = killersudoku.load_killersudoku(file)
        fileout = file.replace('collection', 'collection--emdw').replace('.txt', '.ksd')
        os.makedirs(os.path.dirname(fileout), exist_ok=True)
        
        str_out = ''
        for cage in cages:
            str_out += str(cage[0]) +' ' + ' '.join( str(i)+str(j) for i,j in cage[1]) + '\n'

        open(fileout, 'w').write(
            str_out
        )